In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import Angle

#### Session 1: 1-19-2022

## Helium

In [2]:
#position angle measurements
He_legend = [
    'undeviated', 'violet', 'blue', 'dark green', 'light green', 'orange',
    'red1 secondary', 'red1', 'red2'
]
He_m_legend = [1, 1, 1, 1, 1, 1, 1, 1, 1]
He_lam_legend = ["na", 388.8, 447.1, 501.5, 587.5, 667.8, 686.7, 706.5,
                 728.1]  #wavelength in nanometers
He_left = [
    Angle('0d13m0s'),
    Angle('352d33m0s'),
    Angle('352d7m0s'),
    Angle('351d47m0s'),
    Angle('351d35m0s'),
    Angle('350d7m0s'),
    Angle('349d12m0s'),
    Angle('348d41m0s'),
    Angle('348d0m0s')
]
He_right = [
    Angle('0d13m0s'),
    Angle('7d17m0s'),
    Angle('8d13m0s'),
    Angle('8d30m0s'),
    Angle('8d43m0s'),
    Angle('10d14m0s'),
    Angle('11d7m0s'),
    Angle('11d34m0s'),
    Angle('12d16m0s')
]
N = len(He_legend) - 1

In [3]:
#convert from degrees to radians and define theta as 1/2(alpha1+alpha2)
He_left_rad = np.empty(len(He_legend))
He_right_rad = np.empty(len(He_legend))
theta = np.empty(N)
for j in range(1, len(He_legend)):
    He_left_rad[j] = np.radians(He_left[j].value)
    He_right_rad[j] = np.radians(He_right[j].value)

    theta[j - 1] = ((2 * np.pi - He_left_rad[j]) + He_right_rad[j]) / 2
    print(theta[j - 1])
print(len(theta))

0.1285725882302489
0.1404990047855434
0.1458804366458592
0.14951653925418107
0.17555103392976262
0.19125899719771186
0.1996947552490178
0.21176661590864548
8


In [4]:
d = np.empty(N)
for i in range(0, N):
    d[i] = abs(
        (He_m_legend[i + 1] * He_lam_legend[i + 1]) / (np.sin(theta[i])))
    print("d =", d[i])

d_best = np.sum(d) / N
print("d_best is ", d_best)

d = 3032.320325032994
d = 3192.7226256776726
d = 3449.9702851789307
d = 3944.0096170010856
d = 3823.631337410815
d = 3612.402698389076
d = 3561.523543338367
d = 3464.051955734105
d_best is  3510.0790484703803


In [5]:
#stats and error propagation for d
sig_sys = np.empty(len(theta))
for i in range(0, len(theta)):
    sig_sys[i] = abs((2 * np.pi - He_left_rad[i + 1]) -
                     He_right_rad[i + 1]) / 2  #(alpha1-alpha2)/2
sig_sys_total = np.sum(sig_sys) / N
sig_rand = 1 / 60  #smallest measurement on the vernier scale
sig_theta = ((sig_sys_total)**2 + sig_rand**2)**(0.5)
print("Uncertainty in theta =", sig_theta, "radians")

Uncertainty in theta = 0.016849073569813786 radians


In [6]:
sig_di = np.empty(N)
for i in range(0, N):
    sig_di[i] = (
        (He_m_legend[i + 1] * He_lam_legend[i + 1] * np.cos(theta[i])) /
        (np.sin(theta[i]))**2) * sig_theta
    #print(sig_di[i])
w_i = 1 / sig_di**2  #weight
d_wavg = np.sum(
    (1 / sig_di**2) * (d)) / (np.sum(1 / sig_di**2))  #weighted average
print(d_wavg)
sig_d = (1 / np.sum(1 / sig_di**2))
print(sig_d)

3509.576609358483
14817.79329054617


## Hydrogen

In [7]:
H_legend = ["Undeviated", "Violet", "Teal", "Red"]
H_left = [
    Angle('0d7m0s'),
    Angle('352d42m0s'),
    Angle('351d46m0s'),
    Angle('348d52m0s')
]
H_right = [
    Angle('0d6m0s'),
    Angle('7d26m0s'),
    Angle('8d20m0s'),
    Angle('11d18m0s')
]
H_m = [1, 1, 1, 1]
N_H = len(H_legend)

In [8]:
H_left_rad = np.empty(N_H)
H_right_rad = np.empty(N_H)
theta_H = np.empty(N_H)
for i in range(1, N_H):
    H_left_rad[i] = np.radians(H_left[i].value)
    H_right_rad[i] = np.radians(H_right[i].value)

    theta_H[i] = ((2 * np.pi - He_left_rad[i]) + He_right_rad[i]) / 2
    print(theta_H[i])
print(len(theta_H))

0.1285725882302489
0.1404990047855434
0.1458804366458592
4


In [10]:
n2 = 4
n1 = 2
lam_H = np.empty(N_H)
R_H = np.empty(N_H)
dlam_dd = np.empty(N_H)
dlam_dtheta = np.empty(N_H)
sig_theta_H = np.empty(N_H)
sig_lam_i = np.empty(N_H)

for i in range(1, N_H):
    lam_H[i] = (d_wavg * np.sin(theta_H[i])) / H_m[i]
    R_H[i] = (1 / lam_H[i]) / ((1 / n2**2) - (1 / n1**2))
    dlam_dd[i] = np.sin(theta_H[i]) / H_m[i]
    dlam_dtheta[i] = (d_wavg * np.cos(theta_H[i])) / H_m[i]
    sig_theta_H[i] = ((2 * np.pi - H_left_rad[i]) - H_right_rad[i]) / 2

    sig_lam_i[i] = (((dlam_dd[i]) * sig_d)**2 +
                    ((dlam_dtheta[i]) * sig_theta_H[i])**2)**(0.5)
    print(sig_lam_i[i])

1899.9216779241785
2075.044791427043
2153.9732653946444


## Helium Neon laser

In [ ]:
He_N_left = [Angle('0d 0m 0s'), Angle('349d 15m 0s'), Angle('338d 0m 0s')]
He_N_right = [Angle('0d 0m 0s'), Angle('10d 49m 0s'), Angle('22d 3m 0s')]
print('Helium Neon Laser from the left: ', He_N_left)
print('Helium Neon Laser from the right: ', He_N_right)

In [ ]:
if len(H_left) == len(H_right):
    H_left_deg = np.zeros(len(H_legend))
    H_right_deg = np.zeros(len(H_legend))
    H_diff = np.zeros(len(H_legend))
    for i in range(0, len(H_legend)):
        H_left_deg[i] = abs(360.0 + H_left[0].value - H_left[i].value)
        H_right_deg[i] = abs(H_right[0].value - H_right[i].value)
        He_diff[i] = abs(H_left_deg[i] - H_right_deg[i])
        print("From the LEFT, ", H_legend[i], ' = ', H_left_deg[i])
        print("From the RIGHT, ", H_legend[i], ' = ', H_right_deg[i])
        print("Difference for ", H_legend[i], ' = ', H_diff[i])
        print(" ")
else:
    (print("Lengths don't match"))